In [1]:
import os
import time
from pathlib import Path
from textwrap import dedent

import numpy as np
from pydrake.all import (
    Context,
    DiagramBuilder,
    InverseKinematics,
    MultibodyPlant,
    PiecewisePolynomial,
    RigidTransform,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
)

from manipulation import running_as_notebook
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import LoadScenario, MakeHardwareStation

In [24]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [34]:
scenario_yaml = f"""
model_drivers:
    iiwa: !IiwaDriver
        control_mode: position_only
        hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
directives:
    - add_model:
        name: iiwa
        file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
        default_joint_positions:
            iiwa_joint_1: [-1.57]
            iiwa_joint_2: [0.1]
            iiwa_joint_3: [0]
            iiwa_joint_4: [-1.2]
            iiwa_joint_5: [0]
            iiwa_joint_6: [ 1.6]
            iiwa_joint_7: [0]
    - add_weld:
        parent: world
        child: iiwa::iiwa_link_0
        X_PC:
            translation: [0, -0.5, 0]
            rotation: !Rpy {{ deg: [0, 0, 180] }}
    - add_model:
        name: wsg
        file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
    - add_weld:
        parent: iiwa::iiwa_link_7
        child: wsg::body
        X_PC:
            translation: [0, 0, 0.09]
            rotation: !Rpy {{deg: [90, 0, 90]}}
    - add_model:
        name: block10
        file: package://stackbot/block10.sdf
"""

In [35]:
station = MakeHardwareStation(LoadScenario(data=scenario_yaml), meshcat=meshcat, package_xmls=["../assets/package.xml"])
plant = station.GetSubsystemByName("plant")

In [36]:
builder = DiagramBuilder()
builder.AddSystem(station)

diagram = builder.Build()

In [37]:
# Define the simulator.
simulator = Simulator(diagram)

# run simulation!
meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
# simulator.AdvanceTo(10)
meshcat.StopRecording()
meshcat.PublishRecording()